In [1]:
import pandas as pd

In [2]:
data  =  pd.read_csv("reviews.csv")

In [3]:
data.head(7)

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN
5,2022-07-09 13:20:20,The player controls sometimes disappear for no...,3,7,NaN
6,2022-07-09 13:19:21,I love the selection and the lyrics are provid...,5,0,NaN


In [4]:
data.columns

Index(['Time_submitted', 'Review', 'Rating', 'Total_thumbsup', 'Reply'], dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61594 entries, 0 to 61593
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Time_submitted  61594 non-null  object
 1   Review          61594 non-null  object
 2   Rating          61594 non-null  int64 
 3   Total_thumbsup  61594 non-null  int64 
 4   Reply           216 non-null    object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [6]:
data.shape

(61594, 5)

In [7]:
data.describe()

,Rating,Total_thumbsup
count,61594.000000,61594.000000
mean,3.155989,7.649381
std,1.673285,89.323143
min,1.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,0.000000
75%,5.000000,1.000000
max,5.000000,8195.000000


Now we will chack null values and duplicates

In [8]:
data.isnull().sum()

Time_submitted        0
Review                0
Rating                0
Total_thumbsup        0
Reply             61378
dtype: int64

In [9]:
data.duplicated().sum()

0

Data does not contain any inconsisties

Now we will delete the  some columns which are not reaquired for further process

In [10]:
data.columns

Index(['Time_submitted', 'Review', 'Rating', 'Total_thumbsup', 'Reply'], dtype='object')

In [11]:
data = data.drop(data.columns[[0,3,4]], axis = 1)

In [12]:
data.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1


Now we will check that data has empty entries or not

In [13]:
blanks = []
for index in data["Review"]:
  if index.isspace()== True:
    blanks.append(index)

print(blanks)

[]


In [14]:
data["Rating"].value_counts()

Rating
5    22095
1    17653
4     7842
2     7118
3     6886
Name: count, dtype: int64

now based on the rating col. we will create a new col that will categorize the postive or negative

In [15]:
def rating(rating):
  if rating >= 3 and rating <= 5:
   return "Positive"
  if rating >0 and rating < 3:
   return "Negative"

In [16]:
data["Positive/Negative"] = data["Rating"].apply(rating)

In [17]:
data.head()

,Review,Rating,Positive/Negative
0,"Great music service, the audio is high quality...",5,Positive
1,Please ignore previous negative rating. This a...,5,Positive
2,"This pop-up ""Get the best Spotify experience o...",4,Positive
3,Really buggy and terrible to use as of recently,1,Negative
4,Dear Spotify why do I get songs that I didn't ...,1,Negative


In [18]:
data["Positive/Negative"].value_counts()

Positive/Negative
Positive    36823
Negative    24771
Name: count, dtype: int64

In [19]:
import nltk

In [20]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [21]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [22]:
Sent_analyzer = SentimentIntensityAnalyzer()

In [25]:
data["Scores"] = data["Review"].apply(lambda review: Sent_analyzer.polarity_scores(review))

In [26]:
data.head()

,Review,Rating,Positive/Negative,Scores
0,"Great music service, the audio is high quality...",5,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp..."
1,Please ignore previous negative rating. This a...,5,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co..."
2,"This pop-up ""Get the best Spotify experience o...",4,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co..."
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp..."
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou..."


In [27]:
data.iloc[97]["Scores"]

{'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'compound': 0.4588}

This indicates that the compound value
closer to 1 -> positive ||
 0 -> Neutral ||
 -1  -> Negative


In [28]:
data["compound"] = data["Scores"].apply(lambda score_dict: score_dict["compound"])

In [29]:
data.head()

,Review,Rating,Positive/Negative,Scores,compound
0,"Great music service, the audio is high quality...",5,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp...",0.9211
1,Please ignore previous negative rating. This a...,5,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co...",0.6249
2,"This pop-up ""Get the best Spotify experience o...",4,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co...",0.5859
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",-0.5209
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou...",0.7149


we have seperated compoud col

In [33]:
data["compound_positive/negative"] = data["compound"].apply(lambda score:"Positive" if score >= 0 else "Negative")

In [34]:
data.head()

,Review,Rating,Positive/Negative,Scores,compound,compound_positive/negative
0,"Great music service, the audio is high quality...",5,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp...",0.9211,Positive
1,Please ignore previous negative rating. This a...,5,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co...",0.6249,Positive
2,"This pop-up ""Get the best Spotify experience o...",4,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co...",0.5859,Positive
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",-0.5209,Negative
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou...",0.7149,Positive


We have created col that corresponds to compound

Now we will check the accuracy

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(data["Positive/Negative"], data["compound_positive/negative"])

0.7248270935480728

Model is Ready to check for unknown dataset

In [37]:
example = "I love this product.This is really good"
Sent_analyzer.polarity_scores(example)

{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'compound': 0.8122}

In [38]:
example2 = "I hate this product.This is really bad"
Sent_analyzer.polarity_scores(example2)

{'neg': 0.652, 'neu': 0.348, 'pos': 0.0, 'compound': -0.8173}